In [1]:
!pip install ..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=20541 sha256=ad7cbca57a3aa906729abbf7f24afe92d6e854f74107863f6b1ec765384d2885
  Stored in directory: /tmp/pip-ephem-wheel-cache-_3a30vc7/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1


In [2]:
import torch 

device = torch.device("cuda:3")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_EASY

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    train=True
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    train=False
)
testset.initialize()


In [4]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [5]:
from spuco.group_inference import JTTInference
from spuco.models import MLP
from spuco.util import Trainer
from torch.optim import SGD

model = MLP(3*28*28, len(classes)).to(device)
trainer = Trainer(
    trainset=trainset,
    model=model,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

jtt = JTTInference(trainset, trainer=trainer, num_epochs=1)

In [6]:
group_partition = jtt.infer_groups()

100%|██████████| 938/938 [00:00<00:00, 1594.87batch/s]


In [7]:
for key in group_partition.keys():
    print(key, len(group_partition[key]))

59713
287


In [8]:
from spuco.invariant_train import InvariantTrainsetWrapper
invariant_trainset = InvariantTrainsetWrapper(trainset, group_partition)

In [15]:
from torch.optim import SGD
from spuco.models import MLP
from spuco.invariant_train import GroupDRO, CustomSampleERM

model = MLP(3*28*28, len(classes)).to(device)
group_dro = CustomSampleERM(
    model=model,
    num_epochs=1,
    trainset=invariant_trainset,
    batch_size=64,
    indices=group_partition[(0,0)] + 200 * group_partition[(0,1)]
    optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

ImportError: cannot import name 'CustomSampleERM' from 'spuco.invariant_train' (/home/sjoshi/anaconda3/envs/clip/lib/python3.10/site-packages/spuco/invariant_train/__init__.py)

In [10]:
group_dro.train()

Epoch 0: 100%|██████████| 938/938 [00:03<00:00, 309.27batch/s, accuracy=78.125%, loss=0.817] 


In [11]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 95.50827423167848
Group (0, 1) Test Accuracy: 67.84869976359339
Group (0, 2) Test Accuracy: 60.520094562647756
Group (0, 3) Test Accuracy: 53.191489361702125
Group (0, 4) Test Accuracy: 70.2127659574468
Group (1, 0) Test Accuracy: 53.789731051344745
Group (1, 1) Test Accuracy: 92.66503667481662
Group (1, 2) Test Accuracy: 68.38235294117646
Group (1, 3) Test Accuracy: 59.55882352941177
Group (1, 4) Test Accuracy: 41.666666666666664
Group (2, 0) Test Accuracy: 49.06666666666667
Group (2, 1) Test Accuracy: 79.73333333333333
Group (2, 2) Test Accuracy: 94.93333333333334
Group (2, 3) Test Accuracy: 46.4
Group (2, 4) Test Accuracy: 10.16042780748663
Group (3, 0) Test Accuracy: 62.06030150753769
Group (3, 1) Test Accuracy: 12.090680100755668
Group (3, 2) Test Accuracy: 68.76574307304786
Group (3, 3) Test Accuracy: 96.72544080604534
Group (3, 4) Test Accuracy: 73.29974811083123
Group (4, 0) Test Accuracy: 84.1309823677582
Group (4, 1) Test Accuracy: 0.0
Group (4, 2)

{(0, 0): 95.50827423167848,
 (0, 1): 67.84869976359339,
 (0, 2): 60.520094562647756,
 (0, 3): 53.191489361702125,
 (0, 4): 70.2127659574468,
 (1, 0): 53.789731051344745,
 (1, 1): 92.66503667481662,
 (1, 2): 68.38235294117646,
 (1, 3): 59.55882352941177,
 (1, 4): 41.666666666666664,
 (2, 0): 49.06666666666667,
 (2, 1): 79.73333333333333,
 (2, 2): 94.93333333333334,
 (2, 3): 46.4,
 (2, 4): 10.16042780748663,
 (3, 0): 62.06030150753769,
 (3, 1): 12.090680100755668,
 (3, 2): 68.76574307304786,
 (3, 3): 96.72544080604534,
 (3, 4): 73.29974811083123,
 (4, 0): 84.1309823677582,
 (4, 1): 0.0,
 (4, 2): 21.410579345088163,
 (4, 3): 22.97979797979798,
 (4, 4): 93.68686868686869}

In [12]:
evaluator.worst_group_accuracy

((4, 1), 0.0)

In [13]:
evaluator.average_accuracy

94.51861224047502

In [14]:
evaluator.evaluate_spurious_task()

42.74